In [1]:
import numpy as np
import pandas as pd
import tls_client
from bs4 import BeautifulSoup
from unidecode import unidecode
from betting_functions import get_url_soup
import time

In [2]:
requests = tls_client.Session(
    client_identifier="chrome112",
)

In [60]:
soup = get_url_soup(f'https://www.basketball-reference.com/leagues/NBA_2024_per_game.html')

# Find all <td> elements with the 'data-append-csv' attribute
td_elements = soup.find_all('td', {'data-append-csv': True})

# Create empty lists to store the extracted data
player_codes = []

# Loop through the <td> elements
for td_element in td_elements:
    data_append_csv = td_element['data-append-csv']
    player_name = td_element.find('a').text
    player_codes.append([player_name,data_append_csv])

# Print the extracted data
player_codes

[['Precious Achiuwa', 'achiupr01'],
 ['Bam Adebayo', 'adebaba01'],
 ['Ochai Agbaji', 'agbajoc01'],
 ['Nickeil Alexander-Walker', 'alexani01'],
 ['Grayson Allen', 'allengr01'],
 ['Kyle Anderson', 'anderky01'],
 ['Giannis Antetokounmpo', 'antetgi01'],
 ['Cole Anthony', 'anthoco01'],
 ['OG Anunoby', 'anunoog01'],
 ['Deni Avdija', 'avdijde01'],
 ['Deandre Ayton', 'aytonde01'],
 ['Marvin Bagley III', 'baglema01'],
 ['Patrick Baldwin Jr.', 'baldwpa01'],
 ['LaMelo Ball', 'ballla01'],
 ['Paolo Banchero', 'banchpa01'],
 ['Desmond Bane', 'banede01'],
 ['Harrison Barnes', 'barneha02'],
 ['Scottie Barnes', 'barnesc01'],
 ['RJ Barrett', 'barrerj01'],
 ['Charles Bassey', 'bassech01'],
 ['Nicolas Batum', 'batumni01'],
 ['Malik Beasley', 'beaslma01'],
 ['MarJon Beauchamp', 'beaucma01'],
 ['Patrick Beverley', 'beverpa01'],
 ['Saddiq Bey', 'beysa01'],
 ['Goga Bitadze', 'bitadgo01'],
 ['Anthony Black', 'blackan01'],
 ['Bogdan Bogdanović', 'bogdabo01'],
 ['Devin Booker', 'bookede01'],
 ['Malaki Branham', 

In [61]:
code = player_codes[0][1]
base = f'https://www.basketball-reference.com/players/c/{code}/gamelog/'
# NBA season we will be analyzing
year = '2024'
url = base+year
print(url)
# this is the HTML from the given URL
response1 = requests.get(url)
soup = BeautifulSoup(response1.content)

https://www.basketball-reference.com/players/c/achiupr01/gamelog/2024


In [5]:
def get_headers(soup):
    i = 0
    while i < 40:
        headers = [th.getText() for th in soup.findAll('tr', limit=40)[i].findAll('th')]
        i = i +1
        if headers:
            idx = i
            i = 41
    return headers[1:]


In [62]:
# avoid the first header row
def get_stats_df(soup,headers,player):

    rows = soup.findAll('tr')
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
    player_stats = [x for x in player_stats if len(x) > 4]
    stats = pd.DataFrame(player_stats, columns = headers)
    stats.insert(0, 'Player', player)
    stats.index = range(len(stats))
    return stats

soup = get_url_soup(url)
headers = get_headers(soup)
get_stats_df(soup,headers,player_codes[0][0])

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Precious Achiuwa,1,2023-10-25,24-036,TOR,,MIN,W (+3),0,24:17,...,5,8,0,0,0,2,1,8,4.5,-5


In [63]:
def get_stats(num):
    for person in range(num):

        year = '2024'
        player_id = player_codes[person][1]
        player_name = player_codes[person][0]
        base = f'https://www.basketball-reference.com/players/c/{player_id}/gamelog/'
        url = base+year
        print(url)
        time.sleep(2.95)
        soup = get_url_soup(url)
        
        headers = get_headers(soup)
        stats = get_stats_df(soup,headers,player_name)
        if person == 0:
            data_core = stats
        else:
            data_core = pd.concat([data_core,stats])
        print(person,player_codes[person][0])
    return data_core


In [65]:
test = get_stats(len(player_codes))

https://www.basketball-reference.com/players/c/achiupr01/gamelog/2024
0 Precious Achiuwa
https://www.basketball-reference.com/players/c/adebaba01/gamelog/2024
1 Bam Adebayo
https://www.basketball-reference.com/players/c/agbajoc01/gamelog/2024
2 Ochai Agbaji
https://www.basketball-reference.com/players/c/alexani01/gamelog/2024
3 Nickeil Alexander-Walker
https://www.basketball-reference.com/players/c/allengr01/gamelog/2024
4 Grayson Allen
https://www.basketball-reference.com/players/c/anderky01/gamelog/2024
5 Kyle Anderson
https://www.basketball-reference.com/players/c/antetgi01/gamelog/2024
6 Giannis Antetokounmpo
https://www.basketball-reference.com/players/c/anthoco01/gamelog/2024
7 Cole Anthony
https://www.basketball-reference.com/players/c/anunoog01/gamelog/2024
8 OG Anunoby
https://www.basketball-reference.com/players/c/avdijde01/gamelog/2024
9 Deni Avdija
https://www.basketball-reference.com/players/c/aytonde01/gamelog/2024
10 Deandre Ayton
https://www.basketball-reference.com/pla

In [66]:
with pd.option_context('display.max_rows', None,
                       'display.precision', 3,
                       ):
    display(test.sample(5))

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,OG Anunoby,1,2023-10-25,26-100,TOR,,MIN,W (+3),1,33:48,...,6,6,1,2,3,2,2,20,17.5,+6
0,Kevin Love,1,2023-10-25,35-048,MIA,,DET,W (+1),1,27:43,...,8,10,2,0,0,1,5,13,10.5,+5
0,Zion Williamson,1,2023-10-25,23-111,NOP,@,MEM,W (+7),1,31:43,...,3,7,3,0,0,3,1,23,16.3,-8
0,Franz Wagner,1,2023-10-25,22-059,ORL,,HOU,W (+30),1,27:03,...,1,4,2,1,0,3,1,19,14.0,+10
0,Isaiah Stewart,1,2023-10-25,22-156,DET,@,MIA,L (-1),1,34:36,...,8,14,2,0,0,1,3,14,14.5,+7


In [67]:
test['Player'].unique()

array(['Precious Achiuwa', 'Bam Adebayo', 'Ochai Agbaji',
       'Nickeil Alexander-Walker', 'Grayson Allen', 'Kyle Anderson',
       'Giannis Antetokounmpo', 'Cole Anthony', 'OG Anunoby',
       'Deni Avdija', 'Deandre Ayton', 'Marvin Bagley III',
       'Patrick Baldwin Jr.', 'LaMelo Ball', 'Paolo Banchero',
       'Desmond Bane', 'Harrison Barnes', 'Scottie Barnes', 'RJ Barrett',
       'Charles Bassey', 'Nicolas Batum', 'Malik Beasley',
       'MarJon Beauchamp', 'Patrick Beverley', 'Saddiq Bey',
       'Goga Bitadze', 'Anthony Black', 'Bogdan Bogdanović',
       'Devin Booker', 'Malaki Branham', 'Christian Braun',
       'Mikal Bridges', 'Malcolm Brogdon', 'Dillon Brooks', 'Bruce Brown',
       'Jaylen Brown', 'Kobe Brown', 'Troy Brown Jr.', 'Jalen Brunson',
       'Thomas Bryant', 'Reggie Bullock', 'Alec Burks', 'Jared Butler',
       'Jimmy Butler', 'Kentavious Caldwell-Pope', 'Toumani Camara',
       'Clint Capela', 'Jevon Carter', 'Wendell Carter Jr.',
       'Alex Caruso', 'M

In [68]:
save = test.copy(deep=True)
save.tail(6)

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Christian Wood,1,2023-10-24,28-027,LAL,@,DEN,L (-12),0,15:28,...,3,4,0,0,0,1,1,7,5.2,+2
1,Christian Wood,2,2023-10-26,28-029,LAL,,PHO,W (+5),0,21:24,...,8,10,1,0,0,1,2,7,5.2,+23
0,Delon Wright,1,2023-10-25,31-182,WAS,@,IND,L (-23),0,25:19,...,5,6,6,3,0,0,1,8,12.6,-15
0,Trae Young,1,2023-10-25,25-036,ATL,@,CHO,L (-6),1,36:01,...,1,1,9,1,0,5,3,23,12.3,-11
0,Omer Yurtseven,1,2023-10-25,25-128,UTA,,SAC,L (-16),0,2:25,...,1,1,0,0,0,0,0,0,0.3,+2
0,Ivica Zubac,1,2023-10-25,26-221,LAC,,POR,W (+12),1,25:36,...,7,12,0,0,4,2,3,20,21.0,+25


In [69]:
test['Date'] = pd.to_datetime(test['Date'])
data = test
for i in range(len(data)):
    data.iloc[i,0] = unidecode(data.iloc[i,0]).replace('_',' ')
    #remove special characters

In [71]:
data.head(5)

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Precious Achiuwa,1,2023-10-25,24-036,TOR,,MIN,W (+3),0,24:17,...,5,8,0,0,0,2,1,8,4.5,-5
0,Bam Adebayo,1,2023-10-25,26-099,MIA,,DET,W (+1),1,32:08,...,6,8,3,0,2,2,3,22,18.4,-4
0,Ochai Agbaji,1,2023-10-25,23-188,UTA,,SAC,L (-16),0,19:27,...,3,3,0,0,0,0,2,5,1.7,-11
0,Nickeil Alexander-Walker,1,2023-10-25,25-053,MIN,@,TOR,L (-3),1,22:38,...,4,4,2,1,0,1,3,2,0.3,-17
0,Grayson Allen,1,2023-10-24,28-016,PHO,@,GSW,W (+4),1,20:55,...,1,4,2,0,0,1,2,0,-2.2,-4


In [72]:
data = data.reset_index(drop=True)

In [73]:
len(data['Player'].unique())

336

In [133]:
info = pd.read_csv('player_info.csv',index_col=0)

In [135]:
info['player'] = info['player'].apply(lambda x: x.split('\xa0\xa0(TW)')[0])

In [136]:
info.loc[info.player.str.contains('Butler')].values

array([['MIA', 'Jimmy Butler', 'PF', '6-7', 230, 'September 14, 1989',
        'us', '12', 'Marquette', '2023-10-27'],
       ['WAS', 'Jared Butler', 'SG', '6-3', 195, 'August 25, 2000', 'us',
        '2', 'Baylor', '2023-10-27'],
       ['WAS', 'John Butler', 'C', '7-1', 175, 'December 4, 2002', 'us',
        '1', 'Florida State', '2023-10-27']], dtype=object)

In [137]:
info.head(3)

,team,player,position,height,weight,birth_date,country,years_exp,college,date
0,CHO,P.J. Washington,PF,6-7,230,"August 23, 1998",us,4,Kentucky,2023-10-27
1,CHO,Terry Rozier,SG,6-1,190,"March 17, 1994",us,8,Louisville,2023-10-27
2,CHO,LaMelo Ball,PG,6-7,180,"August 22, 2001",us,3,NaN,2023-10-27


In [94]:
data

{'P.J. Washington': 'PF',
 'Terry Rozier': 'SG',
 'LaMelo Ball': 'PG',
 'Mark Williams': 'C',
 'Brandon Miller': 'SF',
 'Gordon Hayward': 'SF',
 'Nick Richards': 'C',
 'JT Thor': 'PF',
 'Theo Maledon  (TW)': 'PG',
 'Nick Smith Jr.': 'SG',
 'Bryce McGowens': 'SG',
 'Miles Bridges': 'SF',
 'Amari Bailey  (TW)': 'PG',
 'Leaky Black  (TW)': 'SF',
 'Frank Ntilikina': 'PG',
 'Cody Martin': 'SF',
 'James Bouknight': 'SG',
 'Ish Smith': 'PG',
 'Cam Thomas': 'SG',
 'Mikal Bridges': 'SG',
 'Cameron Johnson': 'PF',
 'Dorian Finney-Smith': 'PF',
 'Dennis Smith Jr.': 'PG',
 'Nic Claxton': 'C',
 "Royce O'Neale": 'SF',
 'Spencer Dinwiddie': 'PG',
 "Day'Ron Sharpe": 'C',
 'Ben Simmons': 'PG',
 'Harry Giles': 'C',
 'Trendon Watford': 'PF',
 'Armoni Brooks  (TW)': 'PG',
 'Dariq Whitehead': 'SF',
 'Lonnie Walker IV': 'SG',
 'Noah Clowney': 'PF',
 'Jalen Wilson  (TW)': 'PF',
 'Kevin Durant': 'PF',
 'Josh Okogie': 'SG',
 'Eric Gordon': 'SG',
 'Jusuf Nurkic': 'C',
 'Jordan Goodwin': 'PG',
 'Yuta Watanabe': 

In [139]:
p2pos = {}
info = pd.read_csv('player_info.csv',index_col=0)
info['player'] = info['player'].apply(lambda x: x.split('\xa0\xa0(TW)')[0])


for i in range(len(info)):
    info.iloc[i,1] = unidecode(info.iloc[i,1]).replace('_',' ')

p2pos = {player:pos for player,pos in zip(info['player'].to_list(),info['position'].to_list())}
pos = []
for i in range(len(data)):
    print(data.iloc[i][0])
    pos.append(p2pos[data.iloc[i][0]])
pos

data['Pos'] = pos
data

Precious Achiuwa
Bam Adebayo
Ochai Agbaji
Nickeil Alexander-Walker
Grayson Allen
Grayson Allen
Kyle Anderson
Giannis Antetokounmpo
Cole Anthony
OG Anunoby
Deni Avdija
Deandre Ayton
Marvin Bagley III
Patrick Baldwin Jr.
LaMelo Ball
Paolo Banchero
Desmond Bane
Harrison Barnes
Scottie Barnes
RJ Barrett
Charles Bassey
Nicolas Batum
Malik Beasley
MarJon Beauchamp
Patrick Beverley
Saddiq Bey
Goga Bitadze
Anthony Black
Bogdan Bogdanovic
Devin Booker
Devin Booker
Malaki Branham
Christian Braun
Mikal Bridges
Malcolm Brogdon
Dillon Brooks
Bruce Brown
Jaylen Brown
Kobe Brown
Troy Brown Jr.
Jalen Brunson
Thomas Bryant
Reggie Bullock
Alec Burks
Jared Butler
Jimmy Butler
Kentavious Caldwell-Pope
Toumani Camara
Clint Capela
Jevon Carter
Wendell Carter Jr.
Alex Caruso
Max Christie
Max Christie
Jordan Clarkson
Nic Claxton
Amir Coffey
John Collins
Zach Collins
Mike Conley
Pat Connaughton
Bilal Coulibaly
Robert Covington
Torrey Craig
Jae Crowder
Cade Cunningham
Seth Curry
Stephen Curry
Dyson Daniels
Anth

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Precious Achiuwa,1,2023-10-25,24-036,TOR,,MIN,W (+3),0,24:17,...,8,0,0,0,2,1,8,4.5,-5,C
1,Bam Adebayo,1,2023-10-25,26-099,MIA,,DET,W (+1),1,32:08,...,8,3,0,2,2,3,22,18.4,-4,C
2,Ochai Agbaji,1,2023-10-25,23-188,UTA,,SAC,L (-16),0,19:27,...,3,0,0,0,0,2,5,1.7,-11,SG
3,Nickeil Alexander-Walker,1,2023-10-25,25-053,MIN,@,TOR,L (-3),1,22:38,...,4,2,1,0,1,3,2,0.3,-17,SG
4,Grayson Allen,1,2023-10-24,28-016,PHO,@,GSW,W (+4),1,20:55,...,4,2,0,0,1,2,0,-2.2,-4,SG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,Christian Wood,2,2023-10-26,28-029,LAL,,PHO,W (+5),0,21:24,...,10,1,0,0,1,2,7,5.2,+23,PF
354,Delon Wright,1,2023-10-25,31-182,WAS,@,IND,L (-23),0,25:19,...,6,6,3,0,0,1,8,12.6,-15,PG
355,Trae Young,1,2023-10-25,25-036,ATL,@,CHO,L (-6),1,36:01,...,1,9,1,0,5,3,23,12.3,-11,PG
356,Omer Yurtseven,1,2023-10-25,25-128,UTA,,SAC,L (-16),0,2:25,...,1,0,0,0,0,0,0,0.3,+2,C


In [140]:
data.dropna(inplace=True)
minutes =  data['MP'].to_list()

for i in range(len(minutes)):
    new = minutes[i].split(':')
    res = float(new[0])+(float(new[1])/60)
    minutes[i] = res
data['MP'] = minutes

In [141]:
data.columns = ['Player', 'G', 'Date', 'Age', 'Tm', 'H/A', 'Opp', 'W/L', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'Pos']
home = data['H/A'].values
print(home[0:4])
for i in range(len(home)):
    if home[i] == '@':
        home[i] = 0
    else:
        home[i] = 1
home[0:10]


['' '' '' '@']


array([1, 1, 1, 0, 0, 0, 0, 1, 1, 1], dtype=object)

In [ ]:
home[0:2]

array([1, 0], dtype=object)

In [142]:
data['W/L']

0       W (+3)
1       W (+1)
2      L (-16)
3       L (-3)
4       W (+4)
        ...   
353     W (+5)
354    L (-23)
355     L (-6)
356    L (-16)
357    W (+12)
Name: W/L, Length: 355, dtype: object

In [143]:
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Precious Achiuwa,1,2023-10-25,24-036,TOR,1,MIN,W (+3),0,24.283333,...,8,0,0,0,2,1,8,4.5,-5,C
1,Bam Adebayo,1,2023-10-25,26-099,MIA,1,DET,W (+1),1,32.133333,...,8,3,0,2,2,3,22,18.4,-4,C
2,Ochai Agbaji,1,2023-10-25,23-188,UTA,1,SAC,L (-16),0,19.450000,...,3,0,0,0,0,2,5,1.7,-11,SG
3,Nickeil Alexander-Walker,1,2023-10-25,25-053,MIN,0,TOR,L (-3),1,22.633333,...,4,2,1,0,1,3,2,0.3,-17,SG
4,Grayson Allen,1,2023-10-24,28-016,PHO,0,GSW,W (+4),1,20.916667,...,4,2,0,0,1,2,0,-2.2,-4,SG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,Christian Wood,2,2023-10-26,28-029,LAL,1,PHO,W (+5),0,21.400000,...,10,1,0,0,1,2,7,5.2,+23,PF
354,Delon Wright,1,2023-10-25,31-182,WAS,0,IND,L (-23),0,25.316667,...,6,6,3,0,0,1,8,12.6,-15,PG
355,Trae Young,1,2023-10-25,25-036,ATL,0,CHO,L (-6),1,36.016667,...,1,9,1,0,5,3,23,12.3,-11,PG
356,Omer Yurtseven,1,2023-10-25,25-128,UTA,1,SAC,L (-16),0,2.416667,...,1,0,0,0,0,0,0,0.3,+2,C


In [144]:
data['H/A'] = home
data = data.replace('CHO','CHA')
data = data.replace('PHO','PHX')
data= data.replace('BRK','BKN')

data['W/L'] = data['W/L'].str.extract(r"\(([-+]?\d+)\)").astype(int)
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Precious Achiuwa,1,2023-10-25,24-036,TOR,1,MIN,3,0,24.283333,...,8,0,0,0,2,1,8,4.5,-5,C
1,Bam Adebayo,1,2023-10-25,26-099,MIA,1,DET,1,1,32.133333,...,8,3,0,2,2,3,22,18.4,-4,C
2,Ochai Agbaji,1,2023-10-25,23-188,UTA,1,SAC,-16,0,19.450000,...,3,0,0,0,0,2,5,1.7,-11,SG
3,Nickeil Alexander-Walker,1,2023-10-25,25-053,MIN,0,TOR,-3,1,22.633333,...,4,2,1,0,1,3,2,0.3,-17,SG
4,Grayson Allen,1,2023-10-24,28-016,PHX,0,GSW,4,1,20.916667,...,4,2,0,0,1,2,0,-2.2,-4,SG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,Christian Wood,2,2023-10-26,28-029,LAL,1,PHX,5,0,21.400000,...,10,1,0,0,1,2,7,5.2,+23,PF
354,Delon Wright,1,2023-10-25,31-182,WAS,0,IND,-23,0,25.316667,...,6,6,3,0,0,1,8,12.6,-15,PG
355,Trae Young,1,2023-10-25,25-036,ATL,0,CHA,-6,1,36.016667,...,1,9,1,0,5,3,23,12.3,-11,PG
356,Omer Yurtseven,1,2023-10-25,25-128,UTA,1,SAC,-16,0,2.416667,...,1,0,0,0,0,0,0,0.3,+2,C


In [145]:
data['Date'].max()

Timestamp('2023-10-26 00:00:00')

In [146]:
KM_vals = pd.read_csv('KM_vals.csv',index_col=0)

In [147]:
KM_dict = dict(zip(KM_vals.Player,KM_vals.KM))

In [148]:
data.Player.values[0][0]

'P'

In [149]:
temp = []
for p in data.Player.values:
    if p[0] in KM_dict:
        temp.append(KM_dict[p[0]])
    else:
        temp.append(15)
data['KM'] = temp

In [150]:
data[['FG%','3P%','FT%']] = data[['FG%','3P%','FT%']].replace(5.00,0)

In [151]:
data.to_csv('data.csv',index=False)

In [152]:
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos,KM
0,Precious Achiuwa,1,2023-10-25,24-036,TOR,1,MIN,3,0,24.283333,...,0,0,0,2,1,8,4.5,-5,C,15
1,Bam Adebayo,1,2023-10-25,26-099,MIA,1,DET,1,1,32.133333,...,3,0,2,2,3,22,18.4,-4,C,15
2,Ochai Agbaji,1,2023-10-25,23-188,UTA,1,SAC,-16,0,19.450000,...,0,0,0,0,2,5,1.7,-11,SG,15
3,Nickeil Alexander-Walker,1,2023-10-25,25-053,MIN,0,TOR,-3,1,22.633333,...,2,1,0,1,3,2,0.3,-17,SG,15
4,Grayson Allen,1,2023-10-24,28-016,PHX,0,GSW,4,1,20.916667,...,2,0,0,1,2,0,-2.2,-4,SG,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,Christian Wood,2,2023-10-26,28-029,LAL,1,PHX,5,0,21.400000,...,1,0,0,1,2,7,5.2,+23,PF,15
354,Delon Wright,1,2023-10-25,31-182,WAS,0,IND,-23,0,25.316667,...,6,3,0,0,1,8,12.6,-15,PG,15
355,Trae Young,1,2023-10-25,25-036,ATL,0,CHA,-6,1,36.016667,...,9,1,0,5,3,23,12.3,-11,PG,15
356,Omer Yurtseven,1,2023-10-25,25-128,UTA,1,SAC,-16,0,2.416667,...,0,0,0,0,0,0,0.3,+2,C,15


In [153]:
data.Date.max()

Timestamp('2023-10-26 00:00:00')